In [18]:
import pandas as pd

In [19]:
df_weather = pd.read_csv('./data/weather_data_eng.csv')
df_weather.info()


/var/folders/sg/hlv7r8p13tj6dbrhj1phzxlm0000gn/T/ipykernel_82230/1413179625.py:1: DtypeWarning: Columns (4,7,8,9,11,12,16,24,27,29,36) have mixed types. Specify dtype option on import or set low_memory=False.
  df_weather = pd.read_csv('./data/weather_data_eng.csv')


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2450000 entries, 0 to 2449999
Data columns (total 39 columns):
 #   Column                  Dtype  
---  ------                  -----  
 0   id                      object 
 1   type                    object 
 2   longitude               float64
 3   latitude                float64
 4   TEMP_FLAG               object 
 5   HUMIDEX_FLAG            float64
 6   LOCAL_MONTH             int64  
 7   PRECIP_AMOUNT_FLAG      object 
 8   WINDCHILL_FLAG          object 
 9   WIND_SPEED_FLAG         object 
 10  LOCAL_DATE              object 
 11  RELATIVE_HUMIDITY_FLAG  object 
 12  WEATHER_ENG_DESC        object 
 13  LOCAL_HOUR              int64  
 14  RELATIVE_HUMIDITY       float64
 15  LOCAL_DAY               int64  
 16  STATION_PRESSURE_FLAG   object 
 17  WIND_SPEED              float64
 18  VISIBILITY              float64
 19  LOCAL_YEAR              int64  
 20  UTC_DAY                 int64  
 21  TEMP                    float64

In [ ]:
"""
severe_wind_days = combined_df[combined_df['WIND_SPEED']
                               > WIND_SPEED_THRESHOLD]

winter_storm_days = combined_df[(combined_df['type'] == 'snow') & (
    combined_df['PRECIP_AMOUNT'] >= SNOW_PRECIP_THRESHOLD)]

rainny_storm = combined_df[(combined_df['type'] == 'rain') & (combined_df['PRECIP_AMOUNT'] >= RAIN_PRECIP_THRESHOLD)]

snow_severe = df_weather[(df_weather['WEATHER_ENG_DESC'].str.contains('snow')) & (
    df_weather['PRECIP_AMOUNT'] >= SNOW_PRECIP_THRESHOLD)]
rain_severe = df_weather[(df_weather['WEATHER_ENG_DESC'].str.contains(
    'rain')) & (df_weather['PRECIP_AMOUNT'] >= RAIN_PRECIP_THRESHOLD)]
"""

# Beaufort Wind Scale Table

# =========================

# KM/H DESCRIPTION

- 39 - 49 Strong breeze
- 50 - 61 Near gale
- 89 - 102 Storm


In [41]:
# Convert UTC_DATE to datetime if not already
df_weather['UTC_DATE'] = pd.to_datetime(df_weather['UTC_DATE'])

# Extract hour from UTC_DATE
df_weather['UTC_HOUR'] = df_weather['UTC_DATE'].dt.hour

strong_breeze = range(39, 49)
near_gale = range(50, 61)
gale = range(62, 74)
strong_gale = range(75, 88)
wind_storm = range(89, 102)


strong_breeze_days = df_weather[(df_weather['WIND_SPEED'] >= 39) & (
    df_weather['WIND_SPEED'] <= 49)]
near_gale_days = df_weather[(df_weather['WIND_SPEED'] > 50) & (
    df_weather['WIND_SPEED'] < 61)]
gale_days = df_weather[(df_weather['WIND_SPEED'] > 62) & (
    df_weather['WIND_SPEED'] < 74)]
strong_gale_days = df_weather[(df_weather['WIND_SPEED'] > 75) & (
    df_weather['WIND_SPEED'] < 88)]
wind_storm_days = df_weather[(df_weather['WIND_SPEED'] > 89) & (
    df_weather['WIND_SPEED'] < 102)]

strong_breeze_days.head()
near_gale_days.head()
gale_days.head()
strong_gale_days.head()
wind_storm_days.head()


,id,type,longitude,latitude,TEMP_FLAG,HUMIDEX_FLAG,LOCAL_MONTH,PRECIP_AMOUNT_FLAG,WINDCHILL_FLAG,WIND_SPEED_FLAG,...,UTC_YEAR,UTC_MONTH,STATION_PRESSURE,WIND_DIRECTION,STATION_NAME,PRECIP_AMOUNT,VISIBILITY_FLAG,UTC_DATE,WEATHER_FRE_DESC,UTC_HOUR
1080355,6119M51.2019.2.25.0,Feature,-80.366667,45.033333,NaN,NaN,2,M,NaN,NaN,...,2019,2,96.58,28.0,WESTERN ISLAND (AUT),NaN,NaN,2019-02-25 05:00:00,ND,5
1080356,6119M51.2019.2.25.1,Feature,-80.366667,45.033333,NaN,NaN,2,M,NaN,NaN,...,2019,2,96.67,28.0,WESTERN ISLAND (AUT),NaN,NaN,2019-02-25 06:00:00,ND,6
1080357,6119M51.2019.2.25.2,Feature,-80.366667,45.033333,NaN,NaN,2,M,NaN,NaN,...,2019,2,96.83,28.0,WESTERN ISLAND (AUT),NaN,NaN,2019-02-25 07:00:00,ND,7
1080394,6119M51.2019.2.24.23,Feature,-80.366667,45.033333,NaN,NaN,2,M,NaN,NaN,...,2019,2,96.48,27.0,WESTERN ISLAND (AUT),NaN,NaN,2019-02-25 04:00:00,ND,4
1080396,6119M51.2019.2.25.5,Feature,-80.366667,45.033333,NaN,NaN,2,M,NaN,NaN,...,2019,2,97.51,31.0,WESTERN ISLAND (AUT),NaN,NaN,2019-02-25 10:00:00,ND,10


In [46]:
# max wind speed for each year
filter_year = strong_breeze_days[(strong_breeze_days['UTC_YEAR'] > 2021)]
filter_year
max_strong_breeze = filter_year.groupby(['LOCAL_DATE', 'STATION_NAME', 'longitude', 'latitude', 'UTC_YEAR'])[
    'WIND_SPEED'].max().reset_index()
max_strong_breeze_sorted = max_strong_breeze.sort_values(
    by='WIND_SPEED', ascending=False)
max_strong_breeze_sorted.head()





,LOCAL_DATE,STATION_NAME,longitude,latitude,UTC_YEAR,WIND_SPEED
1054,2022-02-12 05:00:00,WESTERN ISLAND (AUT),-80.366667,45.033333,2022,49.0
3104,2022-06-19 13:00:00,WELCOME ISLAND (AUT),-89.119444,48.369167,2022,49.0
3761,2022-10-13 19:00:00,PORT COLBORNE (AUT),-79.250000,42.866667,2022,49.0
2540,2022-04-14 22:00:00,PORT COLBORNE (AUT),-79.250000,42.866667,2022,49.0
4449,2022-11-20 00:00:00,LAGOON CITY,-79.216667,44.547222,2022,49.0


In [52]:
strong_breeze_stats = strong_breeze_days.groupby(['UTC_YEAR', 'STATION_NAME', 'latitude', 'longitude']).agg(
    days_count=pd.NamedAgg(column='LOCAL_DATE', aggfunc='count'),
    avg_wind_speed=pd.NamedAgg(column='WIND_SPEED', aggfunc='mean'),
    max_wind_speed=pd.NamedAgg(column='WIND_SPEED', aggfunc='max'),
    min_wind_speed=pd.NamedAgg(column='WIND_SPEED', aggfunc='min'),
).reset_index()


strong_breeze_stats

,UTC_YEAR,STATION_NAME,latitude,longitude,days_count,avg_wind_speed,max_wind_speed,min_wind_speed
0,2016,BIG TROUT LAKE,53.816111,-89.891667,3,40.000000,41.0,39.0
1,2016,BIG TROUT LAKE A,53.817778,-89.896944,4,40.000000,41.0,39.0
2,2016,CHATHAM KENT,42.305833,-82.082500,238,41.844538,48.0,39.0
3,2016,COLLINGWOOD,44.500000,-80.216667,7,41.571429,47.0,39.0
4,2016,EARLTON A,47.695000,-79.848889,106,42.971698,48.0,39.0
...,...,...,...,...,...,...,...,...
236,2023,TRENTON A,44.118889,-77.528056,27,41.518519,46.0,39.0
237,2023,VINELAND STATION RCS,43.183333,-79.400000,24,41.916667,49.0,39.0
238,2023,WAWA A,47.966944,-84.786389,4,42.000000,45.0,39.0
239,2023,WELCOME ISLAND (AUT),48.369167,-89.119444,220,43.618182,49.0,39.0


In [38]:
strong_breeze_hourly = strong_breeze_days.groupby(['UTC_YEAR', 'UTC_DATE']).agg(
    max_wind_speed=pd.NamedAgg(column='WIND_SPEED', aggfunc='max')).reset_index()

strong_breeze_hourly.info()

#strong_breeze_hourly.to_csv('./data/strong_breeze_hourly.csv', index=False)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13246 entries, 0 to 13245
Data columns (total 3 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   UTC_YEAR        13246 non-null  int64  
 1   UTC_DATE        13246 non-null  object 
 2   max_wind_speed  13246 non-null  float64
dtypes: float64(1), int64(1), object(1)
memory usage: 310.6+ KB


In [ ]:
storm_severe.head()
storm_severe.to_csv('./storm_severe.csv', index=False)


In [ ]:
snow_severe.head()
snow_severe.to_csv('./snow_severe.csv', index=False)

In [ ]:
weather_eng_desc_unique = df_weather['WEATHER_ENG_DESC'].unique()
# Print each unique value line by line
for value in weather_eng_desc_unique:
    if isinstance(value, str) and 'Snow' in value:
        print(value)

In [ ]:
snow_rows = df_weather[df_weather['WEATHER_ENG_DESC'].notna() & df_weather['WEATHER_ENG_DESC'].str.contains('Snow')]
snow_rows.head()

In [ ]:
snow_rows.to_csv('./snow_rows.csv', index=False)

In [ ]:
rain_rows = df_weather[df_weather['WEATHER_ENG_DESC'].notna() & df_weather['WEATHER_ENG_DESC'].str.contains('Rain')]
rain_rows.head()
rain_rows.to_csv('./rain_rows.csv', index=False)

In [ ]:
max_wind_guts = storm_severe.groupby(['LOCAL_DATE', 'STATION_NAME'])['WIND_SPEED'].max()
max_wind_guts.head()

In [ ]:
stations_with_highest_wind_gusts = storm_severe.groupby(['STATION_NAME'])['WIND_SPEED'].max()
stations_with_highest_wind_gusts.head()

In [ ]:
filter_snow_rows = snow_rows[snow_rows['UTC_YEAR'] > 2021]

stations_with_highest_snow = filter_snow_rows.groupby(['LOCAL_DATE', 'STATION_NAME', 'longitude', 'latitude', 'TEMP'])['PRECIP_AMOUNT'].max()
stations_with_highest_snow.head()

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# Filter data for Toronto stations and years greater than 2021
filter_location_toronto = snow_rows[(snow_rows['STATION_NAME'].str.contains('TORONTO')) & (snow_rows['UTC_YEAR'] > 2021)]

# Group by date, station, longitude, and latitude, then find the maximum temperature for each group
max_snow_toronto = filter_location_toronto.groupby(['LOCAL_DATE', 'STATION_NAME', 'longitude', 'latitude', 'UTC_YEAR'])['TEMP'].max().reset_index()

# Sort the DataFrame by temperature in descending order
max_snow_toronto_sorted = max_snow_toronto.sort_values(by='TEMP', ascending=False)

# Display the first few rows
print(max_snow_toronto_sorted.head())

# Scatter plot
plt.figure(figsize=(10, 5))
sns.scatterplot(data=max_snow_toronto_sorted, x='TEMP', y='UTC_YEAR', palette='viridis', size='TEMP')
plt.title('Temperature vs Snow in Toronto')
plt.xlabel('Temperature (°C)')
plt.ylabel('Year (UTC)')
plt.show()


In [ ]:


# Filter data for Toronto stations and years between 2016 and 2022
filter_location_toronto = snow_rows[(snow_rows['STATION_NAME'].str.contains('TORONTO')) & (snow_rows['UTC_YEAR'].between(2016, 2022))]

# Group by year and calculate the average temperature for each year
average_temp_by_year = filter_location_toronto.groupby('UTC_YEAR')['TEMP'].mean().reset_index()

# Line plot
plt.figure(figsize=(10, 5))
sns.lineplot(data=average_temp_by_year, x='UTC_YEAR', y='TEMP', marker='o', color='green')
plt.title('Average Temperature in Toronto (2016-2022)')
plt.xlabel('Year (UTC)')
plt.ylabel('Average Temperature (°C)')
plt.show()


In [ ]:
filter_wind_by_year_toronto = storm_severe[(storm_severe['STATION_NAME'].str.contains('TORONTO')) & (storm_severe['UTC_YEAR'].between(2016, 2022))]
average_wind_by_year = filter_wind_by_year_toronto.groupby('UTC_YEAR')['WIND_SPEED'].mean().reset_index()

average_wind_by_year.head()

plt.figure(figsize=(10, 5))
sns.lineplot(data=average_wind_by_year, x='UTC_YEAR', y='WIND_SPEED', marker='o', color='red')
plt.title('Average Wind Speed in Toronto (2016-2022)')
plt.xlabel('Year (UTC)')
plt.ylabel('Average Wind Speed (km/h)')
plt.show()